## Import library

In [1]:
import pandas as pd
import numpy as np 

import yfinance as yf
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource


import backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, SignalStrategy

from backtesting.test import SMA, GOOG

backtesting.set_bokeh_output(notebook=True)
            
import itertools 
import logging
logger = logging.getLogger('yfinance')
logger.disabled = True

C:\Users\ASUS\AppData\Local\Temp\ipykernel_15104\3987860494.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
d:\Binance Tradebot\.venv\Lib\site-packages\backtesting\_plotting.py:53: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

## Import utils

In [2]:
from utils.loader import *
from utils.signals import *
from utils.trade import *
from utils.strategy import *

## Plotting sample

In [3]:
test = DataLoader(ticker='NVDA', start='2022-01-01', end='2024-12-31', freq='1d', test_size=0.5)
test.run()
# test     

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


## Backtesting sample

In [11]:
class SmaCross(Strategy):

    short_duration = 10  # Default values, can be overridden
    long_duration = 50

    id = 0

    def init(self):
        price = self.data.Close
        self.ma1 = self.I(EMA, price, self.short_duration)
        self.ma2 = self.I(EMA, price, self.long_duration)

    def next(self):
        # if crossover(self.ma2, self.ma1) and not self.position:
        #     self.sell(size = self.order_size)

        # elif crossover(self.ma1, self.ma2):
        #     self.position.close()

        if crossover(self.ma1, self.ma2):
            self.buy(size = self.order_size, tag=f'Long {self.id}')
            self.id += 1

strategy = EmaCross

short_range = range(3, 4)
long_range = range(10, 11)
combinations = list(itertools.product(short_range, long_range))

high = 0
best = None

for x, y in combinations:
    params = {'short_duration': x, 'long_duration': y}
    obj = BackTrader(data=test.data, strategy=strategy, params=params, commission=0.002, plot=True)
    stats = obj.hypothesis()
    # print(stats.loc['Kelly Criterion']/2)
    # print(strategy.order_size)

    curr = stats.loc['Return [%]']
    # print(curr)
    if curr > high:
        best = params
        high = curr

print(best, high)

{'short_duration': 3, 'long_duration': 10} 1.6015793398004643


In [12]:
obj.train_trades.head()

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,ReturnPct,EntryTime,ExitTime,Duration,Tag,"Entry_EMA(C,3)","Exit_EMA(C,3)","Entry_EMA(C,10)","Exit_EMA(C,10)"
0,40,20,29,24.597094,24.907573,22.137384,27.056803,12.419194,0.012623,2022-02-01,2022-02-14,13 days,Long 0,24.058841,24.527928,24.037484,24.703214
1,37,30,33,26.451008,23.805907,NaN,29.096109,-97.868729,-0.100000,2022-02-15,2022-02-18,3 days,Long 1,25.489468,24.412507,25.020994,24.856612
2,40,50,53,24.459496,26.905445,22.013546,26.905445,97.837982,0.100000,2022-03-16,2022-03-21,5 days,Long 2,23.414338,25.968404,22.923591,24.348074
3,56,100,109,17.824398,17.577488,16.041958,19.606838,-13.826944,-0.013852,2022-05-26,2022-06-09,14 days,Long 3,17.278729,18.379064,17.210820,18.386342
4,58,119,121,17.104092,15.793770,15.393682,18.814501,-75.998657,-0.076609,2022-06-24,2022-06-28,4 days,Long 4,16.674368,16.361225,16.642284,16.549032


In [10]:
obj = BackTrader(data=test.data, strategy=strategy, params=best, commission=0.002)
stats = obj.execute()


In [ ]:
print

<function print(*args, sep=' ', end='\n', file=None, flush=False)>

In [ ]:
stats.loc['Win Rate [%]']

42.10526315789473

In [ ]:
obj.test_trades.head()

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,ReturnPct,EntryTime,ExitTime,Duration,Tag,"Entry_EMA(C,1)","Exit_EMA(C,1)","Entry_EMA(C,4)","Exit_EMA(C,4)"
0,8,13,15,123.520920,119.151591,NaN,148.225104,-34.954627,-0.035373,2024-07-22,2024-07-24,2 days,Long 0,123.520920,114.232353,121.916221,118.999837
1,8,20,21,117.001923,113.491865,NaN,140.402307,-28.080461,-0.030000,2024-07-31,2024-08-01,1 days,Long 1,117.001923,109.193138,112.598601,111.236416
2,9,26,33,104.953789,125.944547,101.805175,125.944547,188.916820,0.200000,2024-08-08,2024-08-19,11 days,Long 2,104.953789,129.979919,103.613233,124.041490
3,7,37,38,129.350021,125.469521,NaN,155.220026,-27.163504,-0.030000,2024-08-23,2024-08-26,3 days,Long 3,129.350021,126.440468,127.002646,126.777775
4,7,39,40,128.280197,124.431791,NaN,153.936237,-26.938841,-0.030000,2024-08-27,2024-08-28,1 days,Long 4,128.280197,125.590607,127.378744,126.663489
